In [ ]:
%matplotlib widget

# Imports
from ctraptools.kymos.detect import Detector
from ctraptools.kymos.interactive import FrameRangeSelector
from lumicks import pylake

import ctraptools.kymos.io as kio
import numpy as np
import tifffile

In [ ]:
# Parameters
filepath = ""
filename = ""
extension = ""
channel = 0

In [ ]:
# Getting kymograph
file = pylake.File(filepath+filename+"."+extension)
kymo = next(iter(file.kymos.values()))

frs = FrameRangeSelector(kymo,px_min=0,px_max=50)

In [ ]:
(range_min,range_max) = frs.get_range()

image = kymo.get_image()[:,range_min:range_max,channel]

fr_str = "fr"+str(range_min)+"-"+str(range_max)
tifffile.imwrite(
            filepath+filename+"_"+fr_str+"_raw.tif", 
            kymo.get_image().astype(np.single)
        )

In [ ]:
# Detecting tracks
detector = Detector(half_t_w=0,peak_det_thresh = 12, n_max=5, c_ub=6, c_def=4, max_dist=6,max_frame_gap=50, min_track_length = 100, min_track_density=0.5, track_heritage_weight=100)
tracks = detector.detect(image)

In [ ]:
# Saving files
kio.write_peak_traces(tracks,filepath+filename+"_"+fr_str+'_C'+str(channel)+"_traces")
kio.save_overlay(tracks,image,filepath+filename+"_"+fr_str+'_C'+str(channel)+"_overlay")


In [ ]:
link_dist = 2

for track in tracks.values():
    prob = track.calculate_stationary_probability(image.shape)

    tifffile.imwrite(filepath+filename+"_"+fr_str+"_track"+str(track.ID)+"_prob.tif", prob.astype(np.single))
    